In [11]:
import os
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
# Import rasterio
import rasterio
from rasterio.mask import mask
from rasterio.plot import show
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import pathlib
from shapely.geometry import box
from PIL import Image

In [12]:
def process_image(vector_file, raster_file, output_folder):
    
    # Read the boundary
    boundary = gpd.read_file(vector_file)
 # Clip the raster
    with rasterio.open(raster_file) as src:
        out_image, out_transform = mask(src, boundary.geometry, crop=True)
        out_meta = src.meta.copy()
        out_meta.update({
            "driver": "GTiff",
            "height": out_image.shape[1],
            "width": out_image.shape[2],
            "transform": out_transform
        })

    
        # Save the clipped raster
    base_name = os.path.basename(raster_file)
    clipped_raster_file = os.path.join(output_folder, f"clipped_{base_name}")
    with rasterio.open(clipped_raster_file, 'w', **out_meta) as dest:
        dest.write(out_image)

        # Process the clipped raster
    with rasterio.open(clipped_raster_file) as src:
        raster_data = src.read(1)
        transform = src.transform
        rows, cols = raster_data.shape

    polygons = []
    values = []

    for row in range(rows):
        for col in range(cols):
            value = raster_data[row, col]
            if value > 0:
            # Get the (x, y) coordinates for the cell corners
            # Use polygons instead of points: Instead of representing each cell as a point, 
            # you could represent it as a small polygon (square). This would create a continuous grid-like representation:
            
                x1, y1 = rasterio.transform.xy(transform, row, col)
                x2, y2 = rasterio.transform.xy(transform, row+1, col+1)
            # Create a polygon for the cell
                cell = box(x1, y1, x2, y2)
                polygons.append(cell)
                values.append(value)

    # Create a GeoDataFrame
    gdf = gpd.GeoDataFrame({'geometry': polygons, 'value': values})
# Use the original CRS of the raster data
    gdf.crs = src.crs
    gdf['density_quantile'] = pd.qcut(gdf['value'], q=40)
# gdf['temp'] = pd.cut(gdf['value'], bins=bins, labels=labels)

# Plot the GeoDataFrame
    f, ax = plt.subplots(1, 1, figsize=(14,8))
    gdf.plot(column='density_quantile', ax=ax, cmap='inferno', markersize=1, alpha=0.7, legend=False)
    f.patch.set_facecolor('black')
    ax.set_facecolor('black')
    
    ax.set_title(f'Night Light of Bangladesh {os.path.splitext(base_name)[0]}', fontsize=14, color='darkcyan')
    plt.tight_layout()

    # Save the plot
    plot_file = os.path.join(output_folder, f"{os.path.splitext(base_name)[0]}_NL.png")
    plt.savefig(plot_file, dpi=300)
    plt.close()

    return plot_file

def create_gif(image_files, output_folder):
    images = [Image.open(image) for image in image_files]
    gif_path = os.path.join(output_folder, 'output.gif')
    
    # Save as GIF
    images[0].save(gif_path, save_all=True, append_images=images[1:], duration=1000, loop=0)
    print(f"GIF created successfully and saved as {gif_path}")

def main():
    # Set up paths
    input_folder = input("Enter the path to the folder containing the images: ")
    output_folder = input("Enter the path to save the processed images, plots, and GIF: ")
    vector_file = input("Enter the path to the vector file containing the boundary: ")
    
    # Create output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)
    
    # Process all raster files in the input folder
    processed_images = []
    for filename in os.listdir(input_folder):
        if filename.endswith(('.tif', '.tiff')):  # Add more extensions if needed
            raster_file = os.path.join(input_folder, filename)
            print(f"Processing {filename}...")
            plot_file = process_image(vector_file, raster_file, output_folder)
            processed_images.append(plot_file)
            print(f"Finished processing {filename}")

    # Create GIF from processed images
    if len(processed_images) > 1:
        create_gif(processed_images, output_folder)
    else:
        print("Not enough images to create a GIF. At least 2 images are required.")

if __name__ == "__main__":
    main()


Enter the path to the folder containing the images:  E:\GeospatialDataEssentials\GeospatialDataEssentials\exercise\data2
Enter the path to save the processed images, plots, and GIF:  E:\GeospatialDataEssentials\GeospatialDataEssentials\exercise\data2\map
Enter the path to the vector file containing the boundary:  E:\GeospatialDataEssentials\GeospatialDataEssentials\exercise\data2\bd.gpkg


Processing 2013.tif...
Finished processing 2013.tif
Processing 2014.tif...
Finished processing 2014.tif
Processing 2015.tif...
Finished processing 2015.tif
Processing 2016.tif...
Finished processing 2016.tif
Processing 2017.tif...
Finished processing 2017.tif
Processing 2018.tif...
Finished processing 2018.tif
Processing 2019.tif...
Finished processing 2019.tif
Processing 2020.tif...
Finished processing 2020.tif
Processing 2021.tif...
Finished processing 2021.tif
Processing 2022.tif...
Finished processing 2022.tif
Processing 2023.tif...
Finished processing 2023.tif
Processing 2024.tif...
Finished processing 2024.tif
GIF created successfully and saved as E:\GeospatialDataEssentials\GeospatialDataEssentials\exercise\data2\map\output.gif


In [ ]:
E:\GeospatialDataEssentials\GeospatialDataEssentials\exercise\data2

In [ ]:
E:\GeospatialDataEssentials\GeospatialDataEssentials\exercise\data2\map
